Training set: MIT-BIH Normal Sinus Rhythm Database (nsrdb)

Number of records: 18 long-term ECG recordings

Duration: Up to 24 hours

Sampling frequency: 128 Hz

Channels: 2 leads

Description: Participants were considered healthy, with no significant arrhythmias

Format: WFDB (.hea + .dat)

In [21]:
import wfdb
import numpy as np
import os


In [22]:
def load_training_data(folder_path, segment_length=1280):
    all_segments = []

    for file in os.listdir(folder_path):
        if file.endswith(".hea"):
            record_path = os.path.join(folder_path, os.path.splitext(file)[0])
            try:
                record = wfdb.rdrecord(record_path)
                signal = record.p_signal[:, 0]
                for i in range(0, len(signal) - segment_length, segment_length):
                    segment = signal[i:i+segment_length]
                    all_segments.append(segment)
            except Exception as e:
                print(f"Could not load {record_path}: {e}")

    if not all_segments:
        raise ValueError(f"No training data found in folder: {folder_path}")

    return np.array(all_segments)


In [23]:
def preprocess_signal(segments):
    # Normalization to range [-1, 1]
    normalized = (segments - np.mean(segments, axis=1, keepdims=True)) / np.std(segments, axis=1, keepdims=True)
    return normalized[..., np.newaxis]


In [24]:
from tensorflow.keras import layers, models

def build_cnn_model(input_shape):
    model = models.Sequential([
        layers.Conv1D(32, 5, activation='relu', input_shape=input_shape),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 5, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [25]:
segments = load_training_data("training/", segment_length=3000)
X_train = preprocess_signal(segments)
y_train = np.ones((X_train.shape[0], 1)) 

model = build_cnn_model(X_train.shape[1:])
model.fit(X_train, y_train, epochs=10, batch_size=32)

ValueError: No training data found in folder: training/